In [17]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI


# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI()
# 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

In [18]:
# 대화 메모리 생성
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history"
)

# 메모리 초기화 
memory.load_memory_variables({})

{'chat_history': []}

In [19]:
# runnable 생성
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history")
)

In [20]:
# chain 생성
chain = runnable | prompt | model 

In [21]:
question = "안녕? 내 이름은 나래야."
response = chain.invoke({"input": question})
print(response)

content='안녕하세요, 나래님! 만나서 반가워요. 무엇을 도와드릴까요?' response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 29, 'total_tokens': 64}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7761da4d-b977-4229-a6e2-b574bc0e1ac3-0'


In [23]:
# 메모리 저장하기 
memory.save_context(
    {"input": question}, {"output": response.content}
)

# 메모리 출력
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='안녕? 내 이름은 나래야.'),
  AIMessage(content='안녕하세요, 나래님! 만나서 반가워요. 무엇을 도와드릴까요?')]}

In [24]:
question = "내 이름을 기억해?"
response = chain.invoke({"input": question})
print(response)

content='네, 나래님의 이름을 기억하겠습니다. 어떤 도움이 필요하신가요?' response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 80, 'total_tokens': 115}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-d4d62113-8d2b-4db2-84ee-5fe85f16b713-0'


---

In [25]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='안녕? 내 이름은 나래야.'),
  AIMessage(content='안녕하세요, 나래님! 만나서 반가워요. 무엇을 도와드릴까요?')]}